### Import libraries

In [ ]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import time
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import os

In [ ]:
ct = datetime.datetime.now()
print("current time:-", ct)

### Get companies and reformat name

In [ ]:
#Get list of company names and replace space,& for URL
df_company = pd.read_excel('../../DataSet/CompanyList_Original.xlsx')
all_names = df_company.Name.unique()
print('Total companies : ' + str(len(all_names)))

#Get list of company names and replace space,& for URL
df = pd.read_excel('../../DataSet/Final.xlsx')
scrapped_names = df.company.unique()
print('Companies scraped : ' + str(len(scrapped_names)))

rem_names = [c for c in all_names if c not in scrapped_names]

In [ ]:
import random
sample = random.sample(rem_names, 200)
print(sample)

In [ ]:
#Get list of company names and replace space,& for URL
name_url = []
for i in sample:
    k = i.replace(" ","+").replace("&","%26").replace("'","%27")
    name_url.append(k)

### Functions to extract data from Reuters

In [ ]:
#Check to see if the element exists
def check_exists_by_xpath(xpath):
    try:
        driver.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True

In [ ]:
#Static Xpaths
loadMoreXpath = str("//div[@class='search-result-more']/div[@class='search-result-more-txt']")
loadNoMoreXpath = str("//div[@class='search-result-more']/div[@class='search-result-more-txt search-result-no-more']")

In [ ]:
def extract_to_excel(excel_path, companies_list, start_index, end_index):
    writer = pd.ExcelWriter(excel_path, engine='openpyxl')
    for company in companies_list[start_index:end_index]:
        #Give chrome driver path
        chromedriver = "/usr/local/bin/chromedriver"
        os.environ["webdriver.chrome.driver"] = chromedriver
        driver2 = webdriver.Chrome(chromedriver)
        url = "https://www.reuters.com/search/news?blob="+company+"&sortBy=date&dateRange=pastYear"    
        driver2.get(url)
        
        is_valid = True
        for count in range(100):
            value = check_exists_by_xpath(loadNoMoreXpath)
            if value == True:
                break
            else:
                try:
                    loadMoreButton = driver2.find_element_by_xpath(loadMoreXpath)
                except:
                    is_valid = False
                    break
                loadMoreButton.click()
                time.sleep(8)
                count=count+1
                print("CLICKED :"+str(count))
                
        if not is_valid: continue
        
        titles=[]
        links=[]
        excerpts=[]
        timestamps=[]

        news_titles = driver2.find_elements_by_xpath(".//div[@class='search-result-content']/h3[@class='search-result-title']/a[@href]")
        news_links = driver2.find_elements_by_xpath(".//div[@class='search-result-content']/h3[@class='search-result-title']/a[@href]")
        news_excerpts = driver2.find_elements_by_xpath((".//div[@class='search-result-content']/div[@class='search-result-excerpt']"))
        news_timestamps = driver2.find_elements_by_xpath((".//div[@class='search-result-content']/h5[@class='search-result-timestamp']"))

        for title in news_titles:
            titles.append(title.text)

        for link in news_links:
            links.append(link.get_attribute('href'))

        for excerpt in news_excerpts:
            excerpts.append(excerpt.text)

        for timestamp in news_timestamps:
            timestamps.append(timestamp.text)

        df = pd.DataFrame({
            "title": titles,
            "link": links,
            "excerpt": excerpts,
            "timestamp":timestamps
        })    

        clean_article_body = []
        for i in links:
            response=requests.get(i)
            #print(response.content)
            contents=response.content
            alltext = BeautifulSoup(contents, "lxml")
            #print(alltext)

            article_body=[]
            results = alltext.find_all('p', attrs={'class':'Paragraph-paragraph-2Bgue ArticleBody-para-TD_9x'})
            for d in results:
            #if title is not None:
                article_body.append(d.text.strip())
                cleantext = BeautifulSoup(str(article_body), "lxml").text
                clean_txt = re.sub(r'[?|$|.|!]',r'',cleantext)
            clean_article_body.append(clean_txt)

        df_body = pd.DataFrame({
            "link": links,
            "body": clean_article_body
        })    

        dfinal = df.merge(df_body, on="link", how = 'inner')
        company = company.replace("+"," ").replace("%26","&").replace("%27","'")
        dfinal.to_excel(writer, re.sub('[^A-Za-z0-9 _]*', '', company), index=False)
        driver2.close()
        ct = datetime.datetime.now()
        print(company+"current time:-", ct)

    writer.save()

### Extract data in loop

Due to limitations, scraping is done in batches

In [ ]:
for counter in range(5):
    excel_path = "../../../Working/Extracted/Scraped_Data_" + str(counter+1) + ".xlsx"
    end_index = 40 * (counter + 1)
    start_index = end_index - 40
    if start_index != 0: start_index += 1
    extract_to_excel(excel_path, name_url, start_index, end_index)

### Save Data to Excel

In [ ]:
from openpyxl import load_workbook

li = []

for file_counter in range(1,6):
    file_name = '../../../Working/Extracted/Scraped_Data_' + str(file_counter) + '.xlsx'
    wb = load_workbook(file_name, read_only=True)
    print(file_name)
    for sheet_name in wb.sheetnames:
        try:
            df = pd.read_excel(file_name, sheet_name, index_col=None)
            df.insert(len(df.columns), 'company', sheet_name.strip())
            li.append(df)
        except:
            print('Error occured in -> Filename : "{0}", Sheetname : "{1}"'.format(file_name, sheet_name))
            continue

frame = pd.concat(li, axis=0, ignore_index=True)

In [ ]:
frame.to_excel('../../DataSet/Final.xlsx', index=False)